# POE user analyzing project
###### **Data reference and questions are here: https://www.kaggle.com/datasets/gagazet/path-of-exile-league-statistic**

---
Hi, I'm Angela. 😊 I'm a novice in data analysis.  I'm a novice in data analysis. I'm interested in the video games, mobility, culture, and content industry. I hope to become a data scientist with anomaly detection and anti-cheating. I hope that my analysis will have a positive impact on the world at least a little bit.  

This is my first Kaggle-data analyzing project on my own. I played(?) with this data for almost a month or more freely, and I finish it to start to answer these questions partially. Finally, I can close the analysis joyfully.
I analyzed this data set with my thoughts and what I learned (as if I were a game analyst 😆).  
_ps. English may seem a little awkward because it is not my first language. I'm sorry. And I used the word 'average' more than the word 'mean' because I wanted to communicate more clearly in my hectic writing. 'mean' is so mean to 'average', 'mean' means 'mean' and... just joking._ 🤪

---
##### **Questions for participants**

1. A total number of players in each division, usage of each class in descending order.
2. Some of the players streaming their game (twitch column). Do they play better than people, who does not?
3. Predict chance to be at top 30 in each division, if we are Necromancer. With and without stream.
4. Average number of finished challanges for each division, show division with highest and lowest average challanges.
5. Show dependency between level and class of died characters. Only for HC divisions.


##### **Contents list**
---
1. data remaking (dummy etc)
2. data distribution (all round veiwing)
3. questions : result, processing, add commentary(그외 추가 이야기 및 부족한 것 필요한 것 개선 방향)
4. what I analized with this dataset(my files)

## Importing Data and Libraries
---

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats

# import time as time

import matplotlib.pyplot as plt
import seaborn as sns
from joypy import joyplot

# visualization setting
plt.rcParams['axes.facecolor'] = 'white'

# import tensorflow as tf
# import keras

np.random.seed(0)
# tf.random.set_seed(0)

# version check
print(sns.__version__)
print(pd.__version__)
print(np.__version__)
# print(tf.__version__)
# print(keras.__version__)

data = pd.read_csv("poe_stats.csv")

# if you need
# !pip install joypy

In [ ]:
df = data
df.head(10)

## Data summary

In [ ]:
# options about float

pd.options.display.float_format = '{:,.2f}'.format
df.describe()

In [ ]:
df.describe(include = 'object')

In [ ]:
df.info()

# 1. rankers by ladder
---
##### Q1. A total number of players in each division, usage of each class in descending order.

First I understood that question and wanted to know about each class ratio in each ladder. And it is true maybe.  
(But... I think this question has to be more clear about 'players'. Because there're quite many users who have multiple characters which are on ranked.)  

So I categorized the meaning of 'players' as characters and users. 

## 1-1. ids(characters) and users in each ladder
If we set the meaning of ‘players’ to ‘characters’, the number of unique ids would be the answer. Also, if we set the meaning of it to 'users', the number of unique accounts would be the answer.  
Each division has different numbers of ids(characters) and accounts, and there are some empty values among the 15000.

In [ ]:
# id, rank, account unique numbers

modes = ['Harbinger', 'Hardcore Harbinger', 'SSF Harbinger', 'SSF Harbinger HC']

for i in modes:
    print("* ladder:", i)
    print("  number of unique id:", df[df['ladder'] == i]['id'].nunique())
    print("  number of unique rank:", df[df['ladder'] == i]['rank'].nunique())
    print("  number of unique account:", df[df['ladder'] == i]['account'].nunique())
    print("  id per account:", round(df[df['ladder'] == i]['id'].nunique()/df[df['ladder'] == i]['account'].nunique(), 2))
    print("  max value of the rank:", df[df['ladder'] == i]['rank'].max())
    if i != 'SSF Harbinger HC':
        print('---------------------------------')

## 1-2. account numbers and ratios of each class in each ladder
We can all class and ladder mode put together in one data frame for analysis, and also make them separately to see more comfortably by descending order.  
First, we can compare the number of specific classes's `account` with other ladders easily.  

In [ ]:
# 1. 
# pivot by class and ladder to one Data frame

df_acc_bylad = df.pivot_table(values = 'account', index = 'ladder', columns = 'class', aggfunc = lambda x: len(x.unique()), margins=True, margins_name="total", fill_value = 0)

# transpose to make class as row
df_acc_bylad = df_acc_bylad.sort_values(ascending = False, by = 'ladder').transpose()
df_acc_bylad.reset_index(drop = False, inplace = True)

# drop 'index = 26 'total' row '
df_acc_bylad.drop(index = 26, inplace = True)
df_acc_bylad.sort_values('total', ascending = False)

## 1-3. [Answer] : characters numbers/ratio of each class in each ladder (descending order)
In the second method, we can compare the number of specific classes's `id` by ladder.

In [ ]:
# 안된것

# ladders = ['Harbinger', 'Hardcore Harbinger', 'SSF Harbinger', 'SSF Harbinger HC']
# cnt = 0

# for i in ladders:
#     s1 = df[df['ladder'] == i].groupby(by = 'class').count()['id'].sort_values(ascending = False)
#     s2 = df[df['ladder'] == i].groupby(by = 'class').count()['id'].sort_values(ascending = False)  / df[df['ladder'] == i].shape[0] * 100
#     tmpdf = pd.DataFrame(columns = ['numbers', 'ratio'])
#     tmpdf['numbers'], tmpdf['ratio'] = s1, s2

#     if cnt == 0:
#         df_cl_ratio_bylad = tmpdf
#     else:
#         cnt += 1
#         pd.concat([df_cl_ratio_bylad, tmpdf])

# df_cl_ratio_bylad

In [ ]:
# 2. 
# pivot by class and ladder, and concat them to one Data frame

ladders = ['Harbinger', 'Hardcore Harbinger', 'SSF Harbinger', 'SSF Harbinger HC']
tmps = []

for i in ladders:
    s1 = df[df['ladder'] == i].groupby(by = 'class').count()['id'].sort_values(ascending = False)
    s2 = df[df['ladder'] == i].groupby(by = 'class').count()['id'].sort_values(ascending = False) / df[df['ladder'] == i].shape[0] * 100
    tmpdf = pd.DataFrame(columns = ['numbers', 'ratio'])
    tmpdf['numbers'], tmpdf['ratio'] = s1, s2
    tmps.append(tmpdf)

df_cl_ratio_bylad = pd.concat(tmps, axis = 1)
df_cl_ratio_bylad.sort_values()

In [ ]:
# 3. 
# if you want to see ratio seperated by ladder

# Harbinger
s1 = df[df['ladder'] == 'Harbinger'].groupby(by = 'class').count()['id'].sort_values(ascending = False)
s2 = df[df['ladder'] == 'Harbinger'].groupby(by = 'class').count()['id'].sort_values(ascending = False)  / df[df['ladder'] == 'Harbinger'].shape[0] * 100
tmpdf = pd.DataFrame(columns = ['numbers', 'ratio'])
tmpdf['numbers'], tmpdf['ratio'] = s1, s2
tmpdf.sort_index()

In [ ]:
# Harbinger HC

s1 = df[df['ladder'] == 'Hardcore Harbinger'].groupby(by = 'class').count()['id'].sort_values(ascending = False)
s2 = df[df['ladder'] == 'Hardcore Harbinger'].groupby(by = 'class').count()['id'].sort_values(ascending = False)  / df[df['ladder'] == 'Hardcore Harbinger'].shape[0] * 100
tmpdf = pd.DataFrame(columns = ['numbers', 'ratio'])
tmpdf['numbers'], tmpdf['ratio'] = s1, s2
tmpdf

In [ ]:
# SSF Harbinger 

s1 = df[df['ladder'] == 'SSF Harbinger'].groupby(by = 'class').count()['id'].sort_values(ascending = False)
s2 = df[df['ladder'] == 'SSF Harbinger'].groupby(by = 'class').count()['id'].sort_values(ascending = False)  / df[df['ladder'] == 'SSF Harbinger'].shape[0] * 100
tmpdf = pd.DataFrame(columns = ['numbers', 'ratio'])
tmpdf['numbers'], tmpdf['ratio'] = s1, s2
tmpdf

In [ ]:
# SSF Harbinger HC

s1 = df[df['ladder'] == 'SSF Harbinger HC'].groupby(by = 'class').count()['id'].sort_values(ascending = False)
s2 = df[df['ladder'] == 'SSF Harbinger HC'].groupby(by = 'class').count()['id'].sort_values(ascending = False)  / df[df['ladder'] == 'SSF Harbinger HC'].shape[0] * 100
tmpdf = pd.DataFrame(columns = ['numbers', 'ratio'])
tmpdf['numbers'], tmpdf['ratio'] = s1, s2
tmpdf

## 1-4. Visualization

### 1-4-1. Characters number of each class in each division separately

In [ ]:
df_cha_bylad = df.pivot_table(values = 'id', index = 'ladder', columns = 'class', aggfunc = 'count', margins=True, margins_name="total", fill_value = 0)

# transpose to make class as row
df_cha_bylad = df_cha_bylad.sort_values(ascending = False, by = 'ladder').transpose()
df_cha_bylad.reset_index(drop = False, inplace = True)

# drop 'index = 26 'total' row '
df_cha_bylad.drop(index = 26, inplace = True)
df_cha_bylad.head(3)

In [ ]:
# if you want to see 'characters' number as seperated by ladder (case when 'Harbinger')

plt.subplots(figsize = (10, 6))

plt.xlabel('Class name')
plt.ylabel('Number of id')
plt.ylim(bottom=0, top=4000)
plt.xticks(rotation=45, fontsize=9)

ax = sns.barplot(x = 'class', y = 'Harbinger', data = df_cha_bylad, palette = "rocket")
for p in ax.patches: 
    ax.annotate("%.0f" % p.get_height(), (p.get_x() + p.get_width()/2., p.get_height() - 30), 
       ha='center', va='center', fontsize=10, color='black', xytext=(0, 10), 
       textcoords='offset points') 
plt.tight_layout()
plt.title('Characters in Harbinger division')
plt.show()

In [ ]:
# # if you want to see 'characters' number as seperated by ladder, others 


# # Hardcore Harbinger 
# plt.subplots(figsize = (10, 6))

# plt.xlabel('Class name')
# plt.ylabel('Number of id')
# plt.ylim(bottom=0, top=4000)
# plt.xticks(rotation=45, fontsize=9)

# ax = sns.barplot(x = 'class', y = 'Hardcore Harbinger', data = df_cha_bylad, palette = "rocket")
# for p in ax.patches: 
#     ax.annotate("%.0f" % p.get_height(), (p.get_x() + p.get_width()/2., p.get_height() - 30), 
#        ha='center', va='center', fontsize=10, color='black', xytext=(0, 10), 
#        textcoords='offset points') 
# plt.tight_layout()
# plt.title('Characters in Hardcore Harbinger division')
# plt.show()


# # SSF Harbinger 
# plt.subplots(figsize = (10, 6))

# plt.xlabel('Class name')
# plt.ylabel('Number of id')
# plt.ylim(bottom=0, top=4000)
# plt.xticks(rotation=45, fontsize=9)

# ax = sns.barplot(x = 'class', y = 'SSF Harbinger', data = df_cha_bylad, palette = "rocket")
# for p in ax.patches: 
#     ax.annotate("%.0f" % p.get_height(), (p.get_x() + p.get_width()/2., p.get_height() - 30), 
#        ha='center', va='center', fontsize=10, color='black', xytext=(0, 10), 
#        textcoords='offset points') 
# plt.tight_layout()
# plt.title('Characters in SSF Harbinger division')
# plt.show()


# # SSF Hardcore Harbinger 
# plt.subplots(figsize = (10, 6))

# plt.xlabel('Class name')
# plt.ylabel('Number of id')
# plt.ylim(bottom=0, top=4000)
# plt.xticks(rotation=45, fontsize=9)

# ax = sns.barplot(x = 'class', y = 'SSF Harbinger HC', data = df_cha_bylad, palette = "rocket")
# for p in ax.patches: 
#     ax.annotate("%.0f" % p.get_height(), (p.get_x() + p.get_width()/2., p.get_height() - 30), 
#        ha='center', va='center', fontsize=10, color='black', xytext=(0, 10), 
#        textcoords='offset points') 
# plt.tight_layout()
# plt.title('Characters in SSF Hardcore Harbinger division')
# plt.show()

### 1-4-2. Characters number by division and class 
without number texts.

In [ ]:
# if you want to see all of them

orderlist = list(df_cha_bylad['class'])

plt.rcParams["figure.figsize"] = (8, 16)
plt.xlabel('Class name')
plt.ylabel('Number of id')
plt.ylim(bottom = 0, top = 4000)
plt.xticks(rotation=0, fontsize=10)
sns.set_style('whitegrid')
ax = sns.countplot(data = df, y = 'class',
              palette="rocket", edgecolor = sns.color_palette("dark", 1), hue = 'ladder', order = orderlist) 
plt.tight_layout()
plt.title('Characters in each class by divisions')
plt.show()

## 1-5. Result and conclusion:
We checked the specific number needed. And there are some preferred classes in each ladder.  

In Harbinger, the most common mode, Pathfinder is the most one.  
And Necromancer is the most preferred in Harbinger hardcore mode. (Berserk was on the same level.)  
In SSF Harbinger, the solo mode, the most picked one is Berserker and Necromancer is also preferred too.  
In SSF Harbinger hardcore, the most one is Necromancer and Berserker is also preferred.  

- Necromancer or Berserker are preferred in some modes, but Pathfinder is the most preferred class, only in Harbinger mode.  
- And we may have to consider that all of them(Necromancer and Berserker and Pathfinder) are one-stat classes later. (Each class is among the Intelligence, Strength, and Dexterity stat.)  

◎ As a result, you can find the preferred class in each mode and find out 'the imbalance in performance between classes' or 'the advantageous skills in certain modes' to see if there is a problem with game balance.

# 2. Are streamers better or not?
---
##### Q2. Some of the players streaming their games (twitch column). Do they play better than people who do not stream?

**Before analyzing, I gave shape to the question. This was important to solve this question especially because the ‘better play’ meaning is a little ambiguous.**

1. We have to set the standard of 'play better' means, so I treat the rank of characters to the 'better play'. Maybe there are more criteria like ranking of 'exp' or 'level', 'challenge' contrasts.  
    But I just used the **rank** because they are ranked players, they are already skilled enough to not need to base their rankings on the number of challenges or levels. In addition, there are many variables that affect classes, parties, items, and skill trees to compare the players. And each mode has a different environment. On other criteria other than game modes, we cannot know and use everything in detail.  

2. And there are some cases where one twitch streamer has multiple accounts or characters. And also, one user has multiple characters on a rank chart. (I discovered it when I analyze the dataset before this project.) So I assumed that one account or one twitch id means one user(the one twitch ‘player’ has multiple accounts or characters or the one ‘player’ has multiple characters).  

3. And there're some missing rank values in each ladder. (When I analyzed the data personally, I substituted the missing data by updating it to rank only the data we have. But,) anyway, I used the original rank in this analysis.  

4. Plus, And I decided to use rank, but how should we decide the ability(rank) to represent one user? My words mean, which one is more accurate, the rank average or median of several characters a user has. I thought it would be ideal if the number of data in a user's ranking was more than 30. But the reality wasn't, so we used both for this analysis.  

5. And I set their highest rank as their representative values. Because it's a question of how well they do, so we can treat the best score(rank) as a kind of their performance.

6. sequence
    1. normally get the average from all dataframe
    2. get the average from each player(user)'s average rank and compare
    3. get the average from each player(user)'s median rank and compare
    4. get the average from each player(user)'s highest rank and compare
    5. statistical significance test (Mann Whitney U test)

(You can skip to 2-5.)

In [ ]:
df['twitch'].describe()

In [ ]:
# Not streaming users
df[df['twitch'].isna() == True].describe(include = 'object')

In [ ]:
# Twitch streaming users
df[df['twitch'].isna() == False].describe(include = 'object')

In [ ]:
# you can see how much missing values in rank are there by ladder

print(df[df['ladder'] == 'Harbinger']['rank'].nunique())
print(df[df['ladder'] == 'Hardcore Harbinger']['rank'].nunique())
print(df[df['ladder'] == 'SSF Harbinger']['rank'].nunique())
print(df[df['ladder'] == 'SSF Harbinger HC']['rank'].nunique())

### dividing data

Let's divide the dataset as whether 'twitch' user or not.  
And divide them by person(twitch unique value is first in the twitch column)'s average.  


In [ ]:
# Let's divide the dataset as whether 'twitch' column is null or not

# each id's rank
df_twt = df[df['twitch'].isna() == False]
df_twt_non = df[df['twitch'].isna() == True]

# each user's mean rank
# grouping by player and get the 'average' rank of each player streaming on twitch or not
df_twt_mean = df_twt.pivot_table(index = 'twitch', values = 'rank', aggfunc = 'mean')
df_twt_non_mean = df_twt_non.pivot_table(index = 'account', values = 'rank', aggfunc = 'mean')

# each user's median rank
# grouping by player and get the 'median' rank of each player streaming on twitch or not
df_twt_med = df_twt.pivot_table(index = 'twitch', values = 'rank', aggfunc = 'median')
df_twt_non_med = df_twt_non.pivot_table(index = 'account', values = 'rank', aggfunc = 'median')

# each user's min(highest) rank
# grouping by player and get the 'median' rank of each player streaming on twitch or not
df_twt_min = df_twt.pivot_table(index = 'twitch', values = 'rank', aggfunc = 'min')
df_twt_non_min = df_twt_non.pivot_table(index = 'account', values = 'rank', aggfunc = 'min')

## 2-1. normal average and median of rank simply

normally get the average and median from all dataframe, and checking standard deviation

In [ ]:
# twtich streamers and not streaming users's rank 'average' of each 'id'
print(df_twt['rank'].mean())
print(df_twt_non['rank'].mean())

In [ ]:
# twtich streamers and not streaming users's rank 'median' of each 'id'
print(df_twt['rank'].median())
print(df_twt_non['rank'].median())

It seems that the rank of users who stream is about 500 higher.  
Considering the total ranking is 15,000, the difference of 500 is about one-thirty, so it's not a small difference, but it doesn't seem to be a very big difference.  
And the median showed a bigger difference. 

In [ ]:
# check the standard deviation

print("All users rank : \n", df['rank'].std())
print("Streaming users rank : \n", df_twt['rank'].std())
print("Not-streaming users rank : \n", df_twt_non['rank'].std())

## 2-2. average by account (when we set the average to a representative value)

Get the average values from each player(user)'s average rank. And check the average and median of them.

In [ ]:
# example of users' rank average.
# grouping by twitch and get average 'rank' of each player streaming on twitch
df_twt_mean.sample(5)

In [ ]:
# grouping by twitch and get average 'rank' of each non_streaming players
df_twt_mean.sample(5)

In [ ]:
# get the average of each player's average rank
print(df_twt_mean['rank'].mean())
print(df_twt_non_mean['rank'].mean())

In [ ]:
# get the median of each player's average rank
print(df_twt_mean['rank'].median())
print(df_twt_non_mean['rank'].median())

## 2-3. median rank of each player and get average(when we set the median to a representative value)

Get the average from each player(user)'s median rank

In [ ]:
# example of users' rank median.
# grouping by twitch and get median 'rank' of each player streaming on twitch
df_twt_med.sample(5)

In [ ]:
# grouping by twitch and get median 'rank' of each non_streaming players
df_twt_non_med.sample(5)

In [ ]:
# get the average of each player's median rank
print(df_twt_med['rank'].mean())
print(df_twt_non_med['rank'].mean())

In [ ]:
# get the median of each player's median rank
print(df_twt_med['rank'].median())
print(df_twt_non_med['rank'].median())

## 2-4. highest rank of each player and get average(when we set the min rank to a representative value)

Get the average from each player(user)'s minimum rank

In [ ]:
# example of users' rank min.
# grouping by twitch and get min 'rank' of each player streaming on twitch
df_twt_min.sample(5)

In [ ]:
# grouping by twitch and get min 'rank' of each non_streaming players
df_twt_non_min.sample(5)

This could be a little smaller than other results. But I think it still has the meaning of comparison.

In [ ]:
# get the average of each player's highest(min) rank
print(df_twt_min['rank'].mean())
print(df_twt_non_min['rank'].mean())

In [ ]:
# get the median of each player's highest(min) rank
print(df_twt_min['rank'].median())
print(df_twt_non_min['rank'].median())

If you compare them by the median, it's almost 1000 points difference.

## 2-5. Visualizing each value
For convenience, I will call a user who does not stream as a 'normal' user.  

And I'll show the means at the Mann-Whitney Test, with a confidence interval. 

In [ ]:
dset = [df_twt['rank'], df_twt_non['rank'], 
       df_twt_mean['rank'], df_twt_non_mean['rank'], 
       df_twt_med['rank'], df_twt_non_med['rank'],
       df_twt_min['rank'], df_twt_non_min['rank']]

x = ['Arithmetic mean - All Twitch_user', 'Arithmetic mean - All normal_user', 
     'Mean of rank average - Twitch_user', 'Mean of rank average - normal_user', 
     'Mean of rank median - Twitch_user', 'Mean of rank median - normal_user', 
     'Mean of rank minimum - Twitch_user', 'Mean of rank minimum - normal_user']

y1 = [x.mean() for x in dset]
y2 = [x.median() for x in dset]

tmp1 = pd.DataFrame(columns = x, data = [y1, y2], index = ['Mean', 'Median'])
tmp1.T

##### visualization - median rank and distribution with boxplot

In [ ]:
# Visualization - median rank and distribution
data_frames = [df_twt, df_twt_mean, df_twt_med, df_twt_min]
data_frames_non = [df_twt_non, df_twt_non_mean, df_twt_non_med, df_twt_non_min]

titles = ['Each Group ranks of character', "Player's mean rank", "Player's median rank", "Player's highest rank"]

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for i, ax in enumerate(axes):
    ax.boxplot([data_frames[i]['rank'], data_frames_non[i]['rank']],
               widths=0.5,
               labels=['Streaming', 'Normal'])
    ax.invert_yaxis()
    ax.text(0.8, data_frames[i]['rank'].median(), data_frames[i]['rank'].median(),
            size=13.5, weight='bold', color='#6441a5')
    ax.text(1.8, data_frames_non[i]['rank'].median(), data_frames_non[i]['rank'].median(),
            size=13.5, color='black')
    ax.set_title(titles[i])

plt.show()

## 2-6. Mann Whitney U test: Statistical significance

The T-test is a test about a statistical comparison of the means of two independent samples, as you know. But these are rank scales, so it's hard to treat this data as a normal t-test case. (Of course, it's hard to say that the simple process of calculating the mean and median ‘absolutely’ doesn’t have a problem…) So I decided to use Mann Whitney U Test, which is a nonparametric method.

*I don't think the statistical significance is not significant for this, especially with little size data. The data size is so little so we can't discuss Gaussian distribution or etc. But I think it's meaningful just to check even if it's not used somewhere.*

What I was wondering about was which one I have to use among these to reach a conclusion, `df_twt`, `df_twt_mean`, `df_twt_med`, `df_twt_min`. They have each problem with themselves.

1. First, I think the `df_twt` and `df_twt_non` have a critical problem in that there are multiple characters a single user has. It can't be a real answer to the meaning(intention) of question.  
2. Second, in case of `df_twt_mean`, there is also a problem with the average rank by each player (not one character, but a user). It's also questionable that the average itself represents the user, which makes it easier to conflict with the concept of average because it's a rank, not a score.  
3. Third, in case of `df_twt_med`, I think the median is not bad because it is ‘real’ value. But there’s also a problem. There is a tendency to add or subtract the existence of other details.  
4. `df_twi_min`, indicating the highest performance of each player, can be a good indicator of the problem in the desired, correct direction of this question. However, considering that there is a difference in difficulty between modes(ladder), there is a problem in that it shouldn't be treated as the same rank is the same ability.


After much consideration, we decided to count the number of drops and the degree of drops for Twitch users **who have a lower value than the mean of the normal user for each method**. I would be grateful if I can get advice about this. 😚

### 2-6-1. Normality test and equal-variance checking

In [ ]:
# dset = [df_twt['rank'], df_twt_non['rank'], 
#        df_twt_mean['rank'], df_twt_non_mean['rank'], 
#        df_twt_med['rank'], df_twt_non_med['rank'],
#        df_twt_min['rank'], df_twt_non_min['rank']]

# x = ['Arithmetic mean - All Twitch_user', 'Arithmetic mean - All normal_user', 
#      'Mean of rank average - Twitch_user', 'Mean of rank average - normal_user', 
#      'Mean of rank median - Twitch_user', 'Mean of rank median - normal_user', 
#      'Mean of rank minimum - Twitch_user', 'Mean of rank minimum - normal_user']

i1 = ['Arithmetic Mean', 'Arithmetic Mean', 'Mean of average','Mean of average', 'Mean of median','Mean of median', 'Mean of minimum', 'Mean of minimum']
i2 = ['t', 'n']*4
c1 = ['statistic', 'pval']

pd.options.display.float_format = '{:,.4f}'.format
tmp = pd.DataFrame(index = [i1, i2], columns = c1)

"""=== Normal-test ==="""
for i in range(len(dset)):
    res1 = scipy.stats.normaltest(dset[i])
    tmp.loc[(i1[i], i2[i]), 'statistic'] = res1[0]
    tmp.loc[(i1[i], i2[i]), 'pval'] = res1[1]
tmp

In [ ]:
i1 = ['Arithmetic Mean', 'Mean of average', 'Mean of median', 'Mean of minimum']
c1 = ['statistic', 'pval']
tmp = pd.DataFrame(index = [i1], columns = c1)

"""=== Equal-variance test ==="""
for j in range(0, len(dset), 2):
    res2 = scipy.stats.levene(dset[j], dset[j+1])
    tmp.loc[i1[j//2], 'statistic'] = res2[0]
    tmp.loc[i1[j//2], 'pval'] = res2[1]
tmp

### 2-6-2. Mann-Whitney U Test, T-test

In [ ]:
# Just all data divided twitch column
scipy.stats.mannwhitneyu(x = dset[0], y = dset[1], alternative = 'two-sided', method = 'auto')

And I addedd the T-test with mean dataset and median dataset, because they are composed by represetative values.  
Let’s put in a dataframe.

In [ ]:
i1 = ['Arithmetic Mean', 'Mean of average', 'Mean of median', 'Mean of minimum']
c1 = ['statistic', 'pval']*2
c2 = ['Mann_Whitney_U', 'Mann_Whitney_U', 'T_Test', 'T_Test']
tmp = pd.DataFrame(index = [i1], columns = [c2, c1])

"""=== Mann-Whitney-U Test, T-Test ==="""
for i in range(0, len(dset), 2):
    j = i//2
    res3 = scipy.stats.mannwhitneyu(x = dset[i], y = dset[i+1], alternative = 'two-sided', method = 'auto')
    res4 = scipy.stats.ttest_ind(a = dset[i], b = dset[i+1], equal_var = False, permutations = 0)
    
    tmp.loc[i1[j], (c2[0], c1[0])] = res3[0]
    tmp.loc[i1[j], (c2[0], c1[1])] = res3[1]
    tmp.loc[i1[j], (c2[2], c1[0])] = res4[0]
    tmp.loc[i1[j], (c2[2], c1[1])] = res4[1]
tmp

All p-values are smaller than 0.05, so we can say that 'there's no basis to consider that there are no differences between Twitch streamers and normal players.'.  

Interestingly, there is little difference in statistics in each t-test, whether using means or media. I think it is reasonable to say that whatever we use as a representative value is not very effective in this question.  


### 2-6-3. Confidence interval 
I attach the confidence interval together. We can imagine from this, "What numbers(rank) are mainly distributed in each group".

_code reference: https://jae-eun-ai.tistory.com/48_

In [ ]:
from numpy import array, mean
from scipy.stats import sem, t
def mean_confidence_interval(data, confidence=.95):
    a = 1.0 * array(data)
    n = len(a)
    m,se = mean(a),sem(a)
    h = se*t._ppf((1 + confidence) / 2, n-1)
    return m, m-h, m+h

# dset = [df_twt['rank'], df_twt_non['rank'], 
#        df_twt_mean['rank'], df_twt_non_mean['rank'], 
#        df_twt_med['rank'], df_twt_non_med['rank']]

llist = []
for i in dset:
    j = mean_confidence_interval(i)
    llist.append(j)

In [ ]:
df_ci = pd.DataFrame(data = llist)
df_ci = df_ci.T
df_ci.columns = ['twi', 'nor', 'twi_mean', 'nor_mean', 'twi_median', 'nor_median', 'twi_min', 'nor_min']
df_ci.index = ['m', 'm-h', 'm+h']
df_ci

In [ ]:
# Visualization
plt.figure(figsize = (6,6))
sns.scatterplot(data = df_ci.T, palette = 'rocket')
plt.xticks(rotation = 45)
plt.gca().invert_yaxis()  # reverse the y axis!

# text
colors = ['#6441a5', 'black']
for i in range(0, 8):
    if i%2 ==0:
        plt.text(i+0.1, df_ci.iloc[0, i]+15, s = round(df_ci.iloc[0, i], 2), size = 12, weight = 'bold', color = colors[0])
    else:
        plt.text(i+0.1, df_ci.iloc[0, i]+15, s = round(df_ci.iloc[0, i], 2), size = 12, color = colors[1])

plt.show()

In [ ]:
# Visualization
plt.figure(figsize = (6,6))
sns.scatterplot(data = df_ci.T, palette = 'rocket')
plt.xticks(rotation = 45)
plt.gca().invert_yaxis()  # reverse the y axis!

# text
colors = ['#6441a5', 'black']
for i in range(0, 8):
    if i%2 ==0:
        plt.text(i+0.1, df_ci.iloc[0, i]+15, s = round(df_ci.iloc[0, i], 2), size = 12, weight = 'bold', color = colors[0])
    else:
        plt.text(i+0.1, df_ci.iloc[0, i]+15, s = round(df_ci.iloc[0, i], 2), size = 12, color = colors[1])

plt.show()

### 2-6-4. [Answer] : 
I attach the confidence interval together. We can imagine from this, "What numbers(rank) are mainly distributed in each group".

_code reference: https://jae-eun-ai.tistory.com/48_

## 2-7. "Who does not?"

I set the Twitch users' data frame and I add whether they are better or not in each case.   

If there's not any worse rank in each case, they were dropped in the data frame.  
If not, they remain in the data frame, and the number of cases when they were worse than normal users would be counted.


```python
# Let's divide the dataset as whether 'twitch' column is null or not

# each id's rank
df_twt = df[df['twitch'].isna() == False]
df_twt_non = df[df['twitch'].isna() == True]

# each user's mean rank
# grouping by player and get the 'average' rank of each player streaming on twitch or not
df_twt_mean = df_twt.pivot_table(index = 'twitch', values = 'rank', aggfunc = 'mean')
df_twt_non_mean = df_twt_non.pivot_table(index = 'account', values = 'rank', aggfunc = 'mean')

# each user's median rank
# grouping by player and get the 'median' rank of each player streaming on twitch or not
df_twt_med = df_twt.pivot_table(index = 'twitch', values = 'rank', aggfunc = 'median')
df_twt_non_med = df_twt_non.pivot_table(index = 'account', values = 'rank', aggfunc = 'median')

# each user's min(highest) rank
# grouping by player and get the 'median' rank of each player streaming on twitch or not
df_twt_min = df_twt.pivot_table(index = 'twitch', values = 'rank', aggfunc = 'min')
df_twt_non_min = df_twt_non.pivot_table(index = 'account', values = 'rank', aggfunc = 'min')
```

sequences: 
1. Make all Twitch users list.
2. Get Twitch users who are under-ranked than normal users' mean, in the case of `mean` is the user's representative.  
    And make a column `case_mean`  with value `1`. (we'll use it as a value presenting how much we can say 'the Twitch user is not better than the normal users.')
3. Get Twitch users who are under-ranked than normal users' mean, in the case of `median` is the user's representative.  
    And make a column `case_med`, with the same methods as the previous one.
4. Get Twitch users who are under-ranked than normal users' mean, in the case of `minimum` is the user's representative.  
    And make a column `case_min`, just like the previous one.
5. Delete the Twitch user's twitch id who doesn't have any `1` in new columns. And count the `1` for sort.  
6. See "who does not".

In [ ]:
# 1. All twitch users - the size is the number of all twitch users.
df_q2 = pd.DataFrame(df_twt['twitch'].drop_duplicates())
df_q2.size

In [ ]:
# we'll use this value for comparing.
df_ci

In [ ]:
# 2. In case of the mean is user's representative rank
df_twt_mean.reset_index(inplace = True)
tmp = df_twt_mean[df_twt_mean['rank'] > df_ci.loc['m', 'nor_mean']][['twitch']]
tmp['case_mean'] = 1

# merging 
df_q2 = pd.merge(left = df_q2, right = tmp[['twitch','case_mean']], how = 'left', on = 'twitch')

# 3. In case of the median is user's representative rank
df_twt_med.reset_index(inplace = True)
tmp = df_twt_med[df_twt_med['rank'] > df_ci.loc['m', 'nor_median']][['twitch']]
tmp['case_med'] = 1

# merging 
df_q2 = pd.merge(left = df_q2, right = tmp[['twitch','case_med']], how = 'left', on = 'twitch')

# 4. In case of the median is user's representative rank
df_twt_min.reset_index(inplace = True)
tmp = df_twt_min[df_twt_min['rank'] > df_ci.loc['m', 'nor_min']][['twitch']]
tmp['case_min'] = 1

# merging 
df_q2 = pd.merge(left = df_q2, right = tmp[['twitch','case_min']], how = 'left', on = 'twitch')
df_q2

In [ ]:
# 5-1. deleting
df_q2.dropna(axis = 0, how = 'all', subset = ['case_mean', 'case_med', 'case_min'], inplace = True)

# 5-2. N/A processing
pd.options.display.float_format = '{:,.0f}'.format
df_q2.fillna(0, inplace = True)

# 5-3. counting
df_q2['cnt'] = df_q2['case_mean'] + df_q2['case_med'] + df_q2['case_min']

# 5-4. you can see 'the' Twitch users! (little ambiguous)
df_q2[df_q2['cnt']!= 0].sort_values(by = 'cnt', ascending = False)

In [ ]:
# get list of 'the' Twitch users!

# (1) these Twitch users are little ambiguous to be said 'not better than Normal users'!
twt_list1 = list(df_q2[df_q2['cnt']!= 0]['twitch'])

# (2) more robust methods to judging not good : these Twitch users are certain to be said 'not better than Normal users'!
twt_list2 = list(df_q2[df_q2['cnt']== 3]['twitch'])

# how much are they different?
print(len(twt_list1), len(twt_list2))

In [ ]:
### if you want to see the list of the twitch users, check this : 
# print(twt_list2)

## 2-8. Result and conclusion: Improvement Direction
I wrote some ideas to improve in analyzing with this results.

- (This is the same context we discussed before analyzing this question.) Existing data can define and analyze something new that can index 'ability(or skill)' based on whether `rank` is higher than `challenge` or experience.
    
    However, since this is ‘character’-limited information, we can discuss at last whether there is a more accurate difference if we find a way to create an indicator with all accounts they have. (if we do this, we must have identified information with a person and his accounts)
    
- Since ‘streaming’ is not necessarily only for users who are good at playing, it would be better to analyze users first by selecting the streamers in the Path Of Exile category within Twitch, for getting insight. 